# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e   
5   213.739802 -0.912810  1234  212.826992  214.652612   425.653985  \
6   154.105155 -0.241947  1234  153.863207  154.347102   307.726415   
7   528.421821  0.842385  1234  529.264206  527.579436  1058.528411   
8    33.768864 -0.645184  1234   33.123680   34.414047    66.247359   
9   954.088245  0.142926  1234  954.231172  953.945319  1908.462343   
10  817.678947 -0.905825  1234  816.773122  818.584771  1633.546244   
11  534.416508  0.451281  1234  534.867789  533.965227  1069.735578   
12  259.874465 -0.850923  1234  259.023542  260.725389   518.047083   
13  105.041789 -0.448978  1234  104.592811  105.490768   209.185622   
14  723.643310  0.360393  1234  724.003703  723.282917  1448.007406   

              f  
5    429.305224  
6    308.694205  
7   1055.158872  
8     68.828095  
9   1907.890639  
10  1637.169543  
11  1067.930454  
12   521.450777  
13   210.981536  
14  1446.565833

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -212.826992   214.652612
6   -153.863207   154.347102
7   -529.264206   527.579436
8    -33.123680    34.414047
9   -954.231172   953.945319
10  -816.773122   818.584771
11  -534.867789   533.965227
12  -259.023542   260.725389
13  -104.592811   105.490768
14  -724.003703   723.282917

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -212.826992   214.652612
6   -153.863207   154.347102
7   -529.264206   527.579436
8    -33.123680    34.414047
9   -954.231172   953.945319
10  -816.773122   818.584771
11  -534.867789   533.965227
12  -259.023542   260.725389
13  -104.592811   105.490768
14  -724.003703   723.282917

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-212.82699226,  214.65261203],
       [-153.86320727,  154.34710226],
       [-529.26420551,  527.57943584],
       [ -33.12367964,   34.41404746],
       [-954.23117154,  953.94531929],
       [-816.77312179,  818.5847713 ],
       [-534.86778891,  533.9652271 ],
       [-259.02354168,  260.72538862],
       [-104.59281097,  105.49076782],
       [-724.00370298,  723.28291654]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5     423.653985   -429.305224
6     305.726415   -308.694205
7    1056.528411  -1055.158872
8      64.247359    -68.828095
9    1906.462343  -1907.890639
10   1631.546244  -1637.169543
11   1067.735578  -1067.930454
12    516.047083   -521.450777
13    207.185622   -210.981536
14   1446.007406  -1446.565833

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf